In [2]:
import torch 
import torch.nn.functional as F
import numpy as np
from torch import nn

In [4]:
# this is copy pasted from tutorial
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import unicodedata
import string

all_letters = string.ascii_letters + " .,;'-"
n_letters = len(all_letters) + 1 # Plus EOS marker

def findFiles(path): return glob.glob(path)

# Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

# Read a file and split into lines
def readLines(filename):
    with open(filename, encoding='utf-8') as some_file:
        return [unicodeToAscii(line.strip()) for line in some_file]

# Build the category_lines dictionary, a list of lines per category
category_lines = {}
all_categories = []
for filename in findFiles('../char-rnn-classify/data/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

if n_categories == 0:
    raise RuntimeError('Data not found. Make sure that you downloaded data '
        'from https://download.pytorch.org/tutorial/data.zip and extract it to '
        'the current directory.')

print('# categories:', n_categories, all_categories)
print(unicodeToAscii("O'Néàl"))

# categories: 18 ['Czech', 'German', 'Arabic', 'Japanese', 'Chinese', 'Vietnamese', 'Russian', 'French', 'Irish', 'English', 'Spanish', 'Greek', 'Italian', 'Portuguese', 'Scottish', 'Dutch', 'Korean', 'Polish']
O'Neal


In [51]:
class Model(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_categories):
        super(Model,self).__init__()
        self.hidden_size = hidden_size
        
        self.i2o = nn.Linear(n_categories + input_size + hidden_size, output_size)
        self.i2h = nn.Linear(n_categories + input_size + hidden_size, hidden_size)

        self.o2o = nn.Linear(output_size + hidden_size, output_size)

        self.dropout = nn.Dropout(0.1)
        self.softmax = nn.Softmax(dim=1)
        
    
    def forward(self, category, input, hidden):        
        combined = torch.cat((category, input, hidden), 1)

        out = self.i2o(combined)
        hidden = self.i2h(combined)

        combined = torch.cat((out, hidden), 1)

        out = self.o2o(combined)
        out = self.dropout(out)
        out = self.softmax(out)

        return (out, hidden)
    
    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)


In [6]:
# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

print(letterToTensor('J'))

print(lineToTensor('Roman').size())

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.]])
torch.Size([5, 1, 59])


# Prepare the dataset

1. For each word hot encode it, add 1 special EOS char in the end.
2. Split each word into first n-1 and last n-1 chars.
3. Each batch will be just one word. Feed consecutively each letter and accumulate loss over the word and then call backwards.

In [20]:
def getCategoryIndex(category, all_categories):
    return all_categories.index(category)

def encodeCategory(category, all_categories, all_categories_size):
    index = getCategoryIndex(category, all_categories)
    tensor = torch.zeros(1, all_categories_size)
    tensor[0,index] = 1.0

    return tensor

getCategoryIndex('English',all_categories)

encodeCategory('English',all_categories, len(all_categories))

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [25]:
# labels all_categories 
# dict category_lines

# each tuple in dataset = (direction vec + encoded letter)
dataset = []

all_categories_size = len(all_categories)

for category in all_categories:
    print(category)
    category_encoded = encodeCategory(category, all_categories, all_categories_size)
    for word in category_lines[category]:
        word_begin = word[:-1]
        word_end = word[1:]
        word_begin_tensor = lineToTensor(word_begin)
        word_end_tensor = lineToTensor(word_end)

        dataset.append((category_encoded, word_begin_tensor, word_end_tensor))




Czech
German
Arabic
Japanese
Chinese
Vietnamese
Russian
French
Irish
English
Spanish
Greek
Italian
Portuguese
Scottish
Dutch
Korean
Polish


In [27]:
import random
random.shuffle(dataset)
len(dataset)

20074

In [55]:
model = Model(n_letters, 128, n_letters, len(all_categories))

criterion = nn.CrossEntropyLoss()

In [78]:
n_epochs = 10
learning_rate = 0.0001

for epoch in range(n_epochs):
    for data in dataset:
        direction, letter_input, letter_target = data
        
        model.zero_grad()
        loss = torch.tensor(0.0)
        hidden = model.init_hidden()
        for letter_index in range(letter_input.size()[1]):
            output, hidden = model(direction, letter_input[letter_index], hidden)
            loss += criterion(output, letter_target[letter_index])
        
        loss.backward()
        for p in model.parameters():
            p.data.add(p.grad.data, alpha=-learning_rate)
    print('epoch...... ', epoch, ' loss...... ', loss)


In [76]:
# Just return an output given a line
def evaluate(direction, first_letter):
    with torch.no_grad():
        direction_tensor = encodeCategory(direction,all_categories,all_categories_size)
        letter_tensor = letterToTensor(first_letter)
        hidden = model.init_hidden()

        limit = 10

        outputs = ""
        for i in range(limit):
            output, hidden = model(direction_tensor, letter_tensor, hidden)
            outputs += all_letters[torch.argmax(output)]
            letter_tensor = output

        return outputs

In [77]:
evaluate('Russian','v')

'DDMDDDDDDD'